In [ ]:
# Library Imports

import numpy as np
import pandas as pd

# Visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go

import json

import warnings

warnings.filterwarnings('ignore')
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


In [4]:
import pandas as pd

# Original Data

data = pd.read_csv(r'/content/Mean Temperature Data.csv')


# Display the first few rows of the dataframe to ensure it loaded correctly
print(data.head())

                States Period   1901   1902   1903   1904   1905   1906  \
0  Andaman and Nicobar    Jan  27.68  26.75  26.70  26.02  26.12  27.62   
1  Andaman and Nicobar    Feb  28.50  26.67  27.54  26.49  27.29  27.40   
2  Andaman and Nicobar    Mar  27.03  28.09  27.66  27.55  27.89  26.88   
3  Andaman and Nicobar    Apr  28.52  29.35  29.53  27.72  28.79  29.62   
4  Andaman and Nicobar    May  28.28  28.25  29.10  28.02  27.80  29.67   

    1907   1908  ...   2011   2012   2013   2014   2015   2016   2017   2018  \
0  26.78  26.40  ...  26.53  26.78  26.83  25.88  26.29  26.80  26.95  26.79   
1  26.92  26.29  ...  26.87  27.00  27.73  26.46  26.75  26.86  26.83  26.61   
2  27.04  27.59  ...  26.39  27.53  27.84  27.54  27.86  28.29  27.78  27.63   
3  28.75  29.23  ...  27.94  28.48  28.73  28.98  28.39  29.74  28.31  28.18   
4  27.42  27.70  ...  27.81  27.94  28.40  28.74  28.78  28.60  28.39  27.74   

    2019   2020  
0  27.30  27.56  
1  27.48  27.14  
2  27.88  27.9

In [11]:

# Function for zonal-classification of states
def zones(state):
    if state in ['Chandigarh','Delhi','Haryana','Himachal Pradesh','Punjab','Rajasthan']:
        return 'North'
    elif state in ['Bihar', 'Orissa', 'Jharkhand', 'West Bengal']:
        return 'East'
    elif state in ['Dadra and Nagar Haveli','Daman and Diu','Goa','Gujarat','Maharashtra']:
        return 'West'
    elif state in ['Andhra Pradesh','Karnataka','Kerala','Puducherry','Tamil Nadu','Andaman and Nicobar','Lakshadweep']:
        return 'South'
    elif state in ['Chhattisgarh','Madhya Pradesh','Uttarakhand','Uttar Pradesh']:
        return 'Central'
    elif state in ['Assam','Sikkim','Nagaland','Meghalaya','Manipur','Mizoram','Tripura','Arunachal Pradesh']:
        return 'North East'
    else:
        return None



In [12]:
# Creating a copy of data
df = data.copy()

# Calculate baseline mean relative to the period 1961-1990
df['Baseline mean'] = df.iloc[:, 62:92].mean(axis=1).round(2)

# Temp. Change relative to the baseline mean
df = df.melt(id_vars=['States', 'Period', 'Baseline mean'], var_name='Year', value_name='temp_change')
df['Year'] = df['Year'].astype(int)
df['Zone'] = df['States'].apply(zones)
df['temp_change'] = df['temp_change'] - df['Baseline mean']
df = df[['States', 'Zone', 'Period', 'Year', 'temp_change', 'Baseline mean']]

# Display the first few rows of the modified dataframe
print(df.head())

                States   Zone Period  Year  temp_change  Baseline mean
0  Andaman and Nicobar  South    Jan  1901         1.87          25.81
1  Andaman and Nicobar  South    Feb  1901         2.44          26.06
2  Andaman and Nicobar  South    Mar  1901         0.12          26.91
3  Andaman and Nicobar  South    Apr  1901         0.42          28.10
4  Andaman and Nicobar  South    May  1901         0.59          27.69


In [14]:
# Which ten states/UTs suffered the most from temperature change in the last ten years?
df_c = df[(df['Period'] == 'Annual') & (df['Year'].between(2011, 2020))][['States', 'temp_change']]
df_c = df_c.groupby('States').mean().round(2).nlargest(10, 'temp_change')
df_c.reset_index(inplace=True)
import plotly.express as px

fig = px.bar(df_c, x='States', y='temp_change', text_auto='.2f',
             title='Ten States/UTs which suffered the most from temperature change in the last ten years (2011-2020)',
             color='temp_change',
             color_continuous_scale='teal'
             )

fig.update_layout(
    width=1000,
    height=600,
    template='plotly_dark',
    coloraxis_showscale=False,
    title_x=0.5
)

fig.update_traces(textposition='outside')
fig.update_xaxes(title='States/UTs')
fig.update_yaxes(title='Temperature Change (°C)')

# Display the figure
fig.show()

In [15]:

# Which ten states/UTs suffered the least from temperature change in the last ten years?
df_c = df[(df.Period == 'Annual') & (df.Year.between(2011, 2020))][['States', 'temp_change']]
df_c = df_c.groupby('States').mean().round(2).nsmallest(10, 'temp_change')
df_c.reset_index(inplace=True)
import plotly.express as px

fig = px.bar(df_c, x='States', y='temp_change', text_auto='.2f',
             title='Ten States/UTs which suffered the least from temperature change in the last ten years (2011-2020)',
             color='temp_change',
             color_continuous_scale='teal'
             )

fig.update_layout(
    width=1000,
    height=600,
    template='plotly_dark',
    coloraxis_showscale=False,
    title_x=0.5
)
fig.update_traces(textposition='outside')
fig.update_yaxes(title='Temperature Change (°C)')

fig.show()

In [ ]:
import folium

indian_states = json.load(open('../input/climate-change-in-india-support-data/states_india.geojson', 'r'))
x = df[(df.Period == 'Annual') & (df.Year.between(2011, 2020))]
x = x.groupby('States').mean().round(2)
x.reset_index(inplace=True)

fig = folium.Map(location=[21.9149, 78.0281], zoom_start=4, tiles=None)
folium.Choropleth(
    geo_data=indian_states,
    data=x,
    bins=9,
    highlight=True,
    columns=['States', "temp_change"],
    key_on="feature.properties.st_nm",
    fill_color='RdPu',
    fill_opacity=1,
    legend_name="Temperature Change (°C)",
    nan_fill_color="Grey"
).add_to(fig)

fig.show()

In [29]:

# Zone-wise temperature change in the last decade (2011-2020)
import plotly.express as px

# Ensure 'Zone' is included in the grouping
df_state_mean = df[(df.Year > 2010) & (df.Period == 'Annual')].groupby(['States', 'Zone'])['temp_change'].mean().reset_index()

# Then, group by Zone and calculate the mean again
df_zone_mean = df_state_mean.groupby('Zone')['temp_change'].mean().round(2).sort_values(ascending=False).reset_index()

fig = px.bar(df_zone_mean,
             text='temp_change',
             y='temp_change',
             title='Zone-wise temperature change in the last ten years (2011-2020)',
             color='temp_change',
             color_continuous_scale='teal'
             )

fig.update_layout(
    width=1000,
    height=600,
    template='plotly_dark',
    coloraxis_showscale=False,
    title_x=0.5
)
fig.update_traces(textposition='outside')
fig.update_yaxes(title='Temperature Change (°C)')

fig.show()

In [18]:

# Temperature change trend in India
fig = px.bar(df[(df.States == 'India') & (df.Period == 'Annual')],
             x='Year',
             y='temp_change',
             title='Temperature change in India (1901-2020)',
             color='temp_change',
             color_continuous_scale='ice_r',
             range_color=[-1, 1.7]
             )

fig.update_layout(
    width=1000,
    height=600,
    template='plotly_dark',
    coloraxis_showscale=False,
    title_x=0.5
)
fig.update_xaxes(tickmode='linear', tick0=1901, dtick=10)
fig.update_yaxes(title='Temperature Change (°C)')

fig.show()

In [20]:
# Filter the data
df0 = df[(df.States == 'India') & (df.Year > 1960)]
df1 = df0[df0.Period == 'Winter']
df2 = df0[df0.Period == 'Summer']
df3 = df0[df0.Period == 'Monsoon']
df4 = df0[df0.Period == 'Post Monsoon']

# Initialize the figure
import plotly.graph_objs as go
fig = go.Figure()

# Add trace to the figure
fig.add_trace(go.Scatter(
    x=df4.Year,
    y=df4.temp_change,
    mode='markers',
    name='Post Monsoon'
))

fig.add_trace(go.Scatter(x=df1.Year, y=df1.temp_change,
                         mode='lines',
                         name='Winter'))
fig.add_trace(go.Scatter(x=df2.Year, y=df2.temp_change,
                         mode='markers',
                         name='Summer'))
fig.add_trace(go.Scatter(x=df3.Year, y=df3.temp_change,
                         mode='lines',
                         name='Monsoon'))

fig.add_annotation(
    x=48,
    y=1.97,
    xref="x",
    yref="y",
    text="Hottest Winter (2009, 1.97)",
    xanchor='right',
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#ffffff"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-20,
    ay=10,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#ff7f0e",
    opacity=1
)

fig.add_annotation(
    x=49,
    y=1.96,
    xref="x",
    yref="y",
    text="Hottest Summer (2010, 1.96)",
    xanchor='center',
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#ffffff"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=-15,
    ay=-30,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor="#ff7f0e",
    opacity=1
)

fig.update_layout(
    width=1000,
    height=600,
    template='seaborn',
    paper_bgcolor="rgb(234, 234, 242)",
    title='Season-wise temperature change in India (1961-2020)',
    title_x=0.5,
    title_y=0.95,
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    )
)

fig.update_xaxes(type='category', title='Years')
fig.update_yaxes(title='Temperature Change (°C)')

fig.show()

In [36]:
# The code below is used to design Climate Spiral ,and it takes around 5-6 minutes to execute.

# =========================================== Code Start ===========================================


# from matplotlib.animation import FuncAnimation, writers

# months=["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
# df_c= df[(df.States=='India') & (df.Period.isin(months)) ][['Period','Year','temp_change']]
# df_c.reset_index(drop=True,inplace=True)

# fig=plt.figure(figsize=(8,8))
# ax=plt.subplot(111,projection="polar")

# ax.plot(np.linspace(0, 2*np.pi, 1000), [2.5]*1000, c='white', linewidth=4)
# ax.plot(np.linspace(0, 2*np.pi, 1000), [4]*1000, c='white', linewidth=4)
# ax.plot(np.linspace(0, 2*np.pi, 1000), [4.5]*1000, c='white', linewidth=4)


# ax.set_xticks([])
# ax.set_yticks([])
# ax.set_ylim(0,5.5)
# fig.set_facecolor("#2f3543")
# ax.set_facecolor("#1c1f2e")


# ax.text(3*np.pi/2, 4.1, "1.5° C", color="white", ha='center', fontsize= 12,bbox=dict(facecolor='#1c1f2e', edgecolor='#1c1f2e'))
# ax.text(3*np.pi/2, 4.6, "2.0° C", color="white", ha='center', fontsize= 12,bbox=dict(facecolor='#1c1f2e', edgecolor='#1c1f2e'))

# theta = np.linspace(0, 2*np.pi, 13)

# fig.text(0.08,0.96,"India's Temperature Change (1901-2020) ",color="white",fontsize=25)
# fig.text(0.05,0.06,"Sarfaraz Sheikh",color="white",fontsize=15)
# fig.text(0.05,0.04,"Based on Ed Hawkins's 2017 Visualization",color="white",fontsize=10)
# fig.text(0.7,0.04,"Baseline: 1961-1990",color="white",fontsize=15)


# #add months ring
# months_angles= np.linspace((np.pi/2)+(2*np.pi),np.pi/2,13)
# for i,month in enumerate(months):
#     ax.text(theta[i],6,month,color="white",fontsize=15,ha="center")

# years= df_c.Year.unique()

# def update(i):
#     if i<df_c.index[-1]:

#         for txt in ax.texts:
#             if(txt.get_position()==(0,0)):
#                 txt.set_visible(False)

#         r = df_c.temp_change[i:i+2] +2.5
#         ax.plot(theta[ [i%12,(i+1)%12] ], r, c=plt.cm.summer_r(i//6), linewidth=2)

#         year= years[i//12]
#         ax.text(0,0,year,fontsize=20,color="white",ha="center")
#     else:
#         ax.plot( theta[df_c.index[-1]%12], df_c.loc[len(df_c)-1].temp_change +2.5,color='orange', marker='o', markersize=8)
#     return ax


# anim = FuncAnimation(fig, update, frames=df_c.index , interval=1)



# #Saving the animation (.gif)
# anim.save('climate_spiral.gif', writer='pillow', fps=50)


# #Saving the animation (.mp4)
# Writer= writers['ffmpeg']
# writer= Writer(fps=96, metadata={'artist':'Me'}, bitrate=-1)
# anim.save('climate_spiral.mp4',writer, dpi=250)

# =========================================== Code End ===========================================

 #from ipywidgets import Video

 #Video.from_file("/content/climate_spiral.mp4", width=400, height=400)